# Klasifikasi Teks

## Instalasi

In [1]:
!pip install nltk
!pip install indoNLP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.6 MB/s eta 0:00:00


In [29]:
import os
import pandas as pd #pandas
import numpy as np #numpy
from tqdm.auto import tqdm
import re #regex
import string #string population
from nltk.tokenize import word_tokenize #tokenize
from nltk.corpus import stopwords #stopword
from indoNLP.preprocessing import replace_slang #slank word
from nltk.stem.porter import PorterStemmer #stemming

## Read Dataset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/prosaindata/tugas/ptaLabel.csv')
df.rename(
    columns={"Abstrak": "abstrak", "Label": "label"},
    inplace=True,
)

In [5]:
df

,NPM,Judul,abstrak,Prodi,label
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,RPL
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,NaN
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,Komputasi
...,...,...,...,...,...
850,160411100032,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Investasi saham selama ini memiliki resiko ker...,Teknik Informatika,komputasi
851,160411100182,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Information Retrieval (IR) merupakan pengambil...,Teknik Informatika,PBA
852,160411100077,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Klasifikasi citra merupakan proses pengelompok...,Teknik Informatika,komputasi
853,160411100084,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Identifikasi atribut pejalan kaki merupakan sa...,Teknik Informatika,komputasi


In [6]:
df['label'].value_counts()

komputasi      344
RPL            168
Komputasi       52
PBA             26
KOMPUTASI        2
KOmputasi        1
komputasi        1
komputai         1
klasifikasi      1
Name: label, dtype: int64

## Change Label

In [7]:
def change_class(before, after):
  df.loc[df['label'] == before, 'label'] = after

In [8]:
change_class('RPL', 'rpl')
change_class('Komputasi', 'komputasi')
change_class('PBA', 'pba')
change_class('KOMPUTASI', 'komputasi')
change_class('KOmputasi', 'komputasi')
change_class('komputasi ', 'komputasi')
change_class('komputai', 'komputasi')
change_class('klasifikasi', 'komputasi')

In [9]:
df['label'].value_counts()

komputasi    402
rpl          168
pba           26
Name: label, dtype: int64

## Remove dataset yang NAN

In [10]:
df.isna().sum()

NPM          0
Judul        6
abstrak     26
Prodi        0
label      259
dtype: int64

In [11]:
df = df[['abstrak', 'label']].dropna().reset_index(drop=True)

In [12]:
df.isna().sum()

abstrak    0
label      0
dtype: int64

In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Pre-Processing

In [14]:
class Prepocessing:
  def __init__(self):
    self.listStopword =  set(stopwords.words('indonesian'))
    self.stemmer = PorterStemmer()

  def remove_emoji(self, string): #remove emoji
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

  def remove_unwanted(self, document): #clean text
    # remove user mentions
    document = re.sub("@[A-Za-z0-9_]+"," ", document)
    # remove URLS 
    document = re.sub(r'http\S+', ' ', document)
    # remove hashtags
    document = re.sub("#[A-Za-z0-9_]+","", document)
    # remove emoji's
    document = self.remove_emoji(document)
    # remove punctuation
    document = re.sub("[^0-9A-Za-z ]", "" , document)
    # remove double spaces
    document = document.replace('  '," ")
    return document.strip()
  
  def tokenize(self, text): #tokenize -> memisah kalimat 
    return word_tokenize(text.translate(str.maketrans('', '', string.punctuation)).lower())
  
  def stopWord(self, text): #stopword -> menghapus kata hubung
    return [kata for kata in text if kata not in self.listStopword]
  
  def slank_word(self, text): #slank word -> mengganti kata yang tidak baku
    return [replace_slang(kata) for kata in text]

  def stemming(self, text): #stemming -> mengganti kata menjadi kata dasar
    return " ".join([self.stemmer.stem(kata) for kata in text])

In [15]:
preprocessing = Prepocessing()

## Clean Text

In [16]:
df['clean'] = df['abstrak'].apply(lambda x: preprocessing.remove_unwanted(x))

## Tokenize

In [17]:
df['tokenize'] = df['clean'].apply(lambda x: preprocessing.tokenize(x))

## Stopword

In [18]:
df['stopword'] = df['tokenize'].apply(lambda x: preprocessing.stopWord(x))

## Slank Word

In [19]:
df['slankword'] = df['stopword'].apply(lambda x: preprocessing.slank_word(x))

## Stemming

In [20]:
df['stem'] = df['slankword'].apply(lambda x: preprocessing.stemming(x))

In [21]:
df[['abstrak', 'clean', 'tokenize', 'stopword', 'slankword', 'stem', 'label']]

,abstrak,clean,tokenize,stopword,slankword,stem,label
0,Sistem informasi akademik (SIAKAD) merupaka...,Sistem informasi akademik SIAKAD merupakan sis...,"[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ...",sistem informasi akademik siakad sistem inform...,rpl
1,Berjalannya koneksi jaringan komputer dengan l...,Berjalannya koneksi jaringan komputer dengan l...,"[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, lan...","[berjalannya, koneksi, jaringan, komputer, lan...",berjalannya koneksi jaringan komput lancar gan...,rpl
2,Web server adalah sebuah perangkat lunak serve...,Web server adalah sebuah perangkat lunak serve...,"[web, server, adalah, sebuah, perangkat, lunak...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, berfun...",web server perangkat lunak server berfungsimen...,rpl
3,Seiring perkembangan teknologi yang ada diduni...,Seiring perkembangan teknologi yang ada diduni...,"[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, didunia, mu...","[seiring, perkembangan, teknologi, didunia, mu...",seir perkembangan teknolog didunia muncul tekn...,komputasi
4,Gerak pekerja ada pada game yang memiliki genr...,Gerak pekerja ada pada game yang memiliki genr...,"[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, game, memiliki, genre, rts, r...","[gerak, pekerja, game, memiliki, genre, rts, r...",gerak pekerja game memiliki genr rt realtim st...,komputasi
...,...,...,...,...,...,...,...
591,Investasi saham selama ini memiliki resiko ker...,Investasi saham selama ini memiliki resiko ker...,"[investasi, saham, selama, ini, memiliki, resi...","[investasi, saham, memiliki, resiko, kerugian,...","[investasi, saham, memiliki, resiko, kerugian,...",investasi saham memiliki resiko kerugian dikar...,komputasi
592,Information Retrieval (IR) merupakan pengambil...,Information Retrieval IR merupakan pengambilan...,"[information, retrieval, ir, merupakan, pengam...","[information, retrieval, ir, pengambilan, info...","[information, retrieval, ir, pengambilan, info...",inform retriev ir pengambilan informasi tersim...,pba
593,Klasifikasi citra merupakan proses pengelompok...,Klasifikasi citra merupakan proses pengelompok...,"[klasifikasi, citra, merupakan, proses, pengel...","[klasifikasi, citra, proses, pengelompokan, pi...","[klasifikasi, citra, proses, pengelompokan, pi...",klasifikasi citra prose pengelompokan piksel c...,komputasi
594,Identifikasi atribut pejalan kaki merupakan sa...,Identifikasi atribut pejalan kaki merupakan sa...,"[identifikasi, atribut, pejalan, kaki, merupak...","[identifikasi, atribut, pejalan, kaki, salah, ...","[identifikasi, atribut, pejalan, kaki, salah, ...",identifikasi atribut pejalan kaki salah peneli...,komputasi


In [22]:
df_final = df[['stem', 'label']]

In [23]:
df_final.to_csv('pta_fix.csv', index=False)

In [25]:
stop_words = set(stopwords.words('indonesian'))

In [26]:
df = pd.read_csv('/content/pta_fix.csv')
df

,stem,label
0,sistem informasi akademik siakad sistem inform...,rpl
1,berjalannya koneksi jaringan komput lancar gan...,rpl
2,web server perangkat lunak server berfungsimen...,rpl
3,seir perkembangan teknolog didunia muncul tekn...,komputasi
4,gerak pekerja game memiliki genr rt realtim st...,komputasi
...,...,...
591,investasi saham memiliki resiko kerugian dikar...,komputasi
592,inform retriev ir pengambilan informasi tersim...,pba
593,klasifikasi citra prose pengelompokan piksel c...,komputasi
594,identifikasi atribut pejalan kaki salah peneli...,komputasi


In [30]:
sentences = [word_tokenize(kata) for kata in tqdm(df.stem)]

  0%|          | 0/596 [00:00<?, ?it/s]

## Train Word2Vec Model

In [32]:
model_word2vec = Word2Vec(sentences, vector_size=128, window=5, min_count=3, workers=4, sg=0, hs=0, epochs=1000)

In [33]:
folder = 'w2v/'
path = '/content/drive/MyDrive/prosaindata/prosaindata/' + folder 
os.makedirs(path, exist_ok=True)

In [34]:
model_word2vec.save(path+'text_klasifikasi.w2v')

In [31]:
from gensim.models import Word2Vec

### Save Word2Vec Model

In [35]:
folder = 'w2v/'
path = '/content/drive/MyDrive/prosaindata/prosaindata/' + folder 
os.makedirs(path, exist_ok=True)

In [36]:
model_word2vec.save(path+'text_klasifikasi.w2v')

## Encoding

In [37]:
from gensim.models import Word2Vec

In [38]:
w2v = Word2Vec.load(path+'text_klasifikasi.w2v').wv

In [39]:
def sent_vector(sentence, w2v_model, stopword):
  vecs = [w2v_model[kata.lower()] for kata in (sentence) if kata in w2v_model]
  sent_vec = np.mean(vecs, axis=0)
  return sent_vec

def norm_sent_vector(sentence, w2v_model, stopword):
  vecs = [w2v_model[kata.lower()] for kata in (sentence) if kata not in stopword]
  norm_vecs = [vec / np.linalg.norm(vec) for vec in vecs if np.linalg.norm(vec) > 0]
  sent_vec = np.mean(norm_vecs, axis=0)
  return sent_vec

In [40]:
vecs = [sent_vector(kata, w2v, stop_words) for kata in sentences]
vecs = np.array(vecs)
vecs.shape

(596, 128)